The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

Visually, if we omit the "pixel" prefix, the pixels make up the image like this:

000 001 002 003 ... 026 027
028 029 030 031 ... 054 055
056 057 058 059 ... 082 083
 |   |   |   |  ...  |   |
728 729 730 731 ... 754 755
756 757 758 759 ... 782 783 
The test data set, (test.csv), is the same as the training set, except that it does not contain the "label" column.

Your submission file should be in the following format: For each of the 28000 images in the test set, output a single line containing the ImageId and the digit you predict. For example, if you predict that the first image is of a 3, the second image is of a 7, and the third image is of a 8, then your submission file would look like:

ImageId,Label
1,3
2,7
3,8 
(27997 more lines)
The evaluation metric for this contest is the categorization accuracy, or the proportion of test images that are correctly classified. For example, a categorization accuracy of 0.97 indicates that you have correctly classified all but 3% of the images.

In [1]:
import numpy as np
import pandas as pd
from matplotlib.pylab import plt

In [3]:
train = pd.read_csv('./data/train.csv', delimiter=',', header=0)
test = pd.read_csv('./data/test.csv', delimiter=',', header=0)

In [4]:
# for index, row in train.iterrows():
#    key = row['label']
#    print('Label: ',str(key))
#    data = np.array(row[1:]).reshape((28, 28))
#    plt.imshow(data, cmap=plt.get_cmap('gray'))
#    plt.show()

In [5]:
# train['img'] = train.apply(lambda row: np.array(row[1:]).reshape((28, 28)), axis=1)

# Unfortunately it's not possible to store a 2d array in a Serie of pandas dataframe,
# I would require to use a Panel (i.e. Dataframe of Dataframe, I don't see how to do that with Scikit easily)
# So hopefully neural network model can take a row as input and interpret that as a 28x28 image array

Open questions:
- How to not trip the ML classifier when the digit is in a corner and not in the center ?
- What if colors are inverted ?

Ideas :
- Generate more data by translating data to the left/right/up/down
- Sharpen the image and add to the data with opencv/imagemagick
- scale up or down the data

In [17]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import GridSearchCV

from sklearn.cross_validation import cross_val_score

from mlxtend.classifier import EnsembleVoteClassifier, StackingClassifier

import time

In [7]:
clf_xgb = XGBClassifier( #0.88 - n = 30 - no feature scaling
        n_estimators=30,
        )
clf_rf = RandomForestClassifier( #0.959 - n = 30 - no feature scaling
        n_estimators=30,
        n_jobs=-1,
        )
clf_et = ExtraTreesClassifier( #0.971 - n=1000 - no feature scaling
        n_estimators=1000,
        n_jobs=-1,
        random_state=1
        )
clf_svc = SVC() #0.96 No multiproc, superlong to train (or BLAS issue ?)
clf_lr = LogisticRegression(n_jobs=-1) 
clf_nb = GaussianNB()
clf_kn = KNeighborsClassifier(n_jobs=-1)

stack_et = ExtraTreesClassifier()

clf_stack = StackingClassifier([clf_et, clf_xgb, clf_rf, clf_svc, clf_lr, clf_nb, clf_kn],stack_et)

In [8]:
featurelist = (''.join(['pixel',str(i)]) for i in range(784))
#featurelist generator

In [9]:
mapper = DataFrameMapper(
    [ ([pixel], StandardScaler()) for pixel in featurelist]
)
# Do we need StandardScaler() ? It's the same scale of color for all pixels

In [10]:
pipe = Pipeline([
    ("featurize", mapper),
#    ("extratrees", clf_et)
#   ("xgboost", clf_xgb)
#   ("randomforest",clf_rf)
#   ("supportvector",clf_svc)
#   ("logit",clf_lr) #Best for binary classification so ignore
#   ("gauss",clf_nb) #Naive_Bayes assume strong feature independance
#   ("kn",clf_kn)
    ("stacking", clf_stack)
])

In [39]:
########################Helper functions ################
##### Cross Validation
def crossval():
    # split = ShuffleSplit(n_splits=10) #compat issue with sklearn pandas "object is not iterable"
    cv = cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=-1)
    print("Cross Validation Scores are: ", cv.round(3))
    print("Mean CrossVal score is: ", round(cv.mean(),3))
    print("Std Dev CrossVal score is: ", round(cv.std(),3))
    return cv
def output():
    # predictions = pipe.predict(test)
    result = pd.DataFrame(predictions)
    result.index+=1
    result.index.names=['ImageId']
    result.columns=['Label']
    result.to_csv(time.strftime("%Y-%m-%d_%H%M-") + 'stacking.csv', header=True)

In [12]:
X_train = train
y_train = train['label']

In [13]:
# temp_cv = crossval()

In [14]:
pipe.fit(X_train, y_train)

/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Pipeline(steps=[('featurize', DataFrameMapper(default=False,
        features=[(['pixel0'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['pixel1'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['pixel2'], StandardScaler(copy=True, with_mean=True, with_std=True)), (['pixel3'], S...random_state=None,
           verbose=0, warm_start=False),
          use_probas=False, verbose=0))])

In [40]:
output()